#### Build zoningmods lookup table for PBA50+ and add Draft Blueprint zoning values

In [1]:
import pandas as pd
import numpy as np

Build the PBA50+ zoningmods table using the parcels geography table and "zoningmodcat" column

In [2]:
# read in parcels geography
parcels_geog = pd.read_csv("M:/urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/parcels_geography_2024_01_30.csv")

In [3]:
# add a zoniongmodcat column to parcels geography
parcels_geog['zoningmodcat'] = ''
for col in ["gg_id", "exd_id", "tra_id", "hra_id", "ppa_id", "ugb_id"]:
    # leave this line in only because the PBA50 BAUS code makes strings lowercase
    # and it will ensure the PBA50 strategy inputs will work
    parcels_geog[col] = parcels_geog[col].str.lower()
    parcels_geog[col] = parcels_geog[col].fillna('NA')
    parcels_geog['zoningmodcat'] = parcels_geog['zoningmodcat'] + parcels_geog[col]

In [4]:
# build zoningmods using zoningmodcat values from parcels geography as the primary key
zoningmods = pd.DataFrame({'zoningmodcat': parcels_geog['zoningmodcat'].unique()})

# then add other zoningmods columns
for col in ["gg_id", "exd_id", "tra_id", "hra_id", "ppa_id", "ugb_id", 
            "add_bldg", "drop_bldg", "dua_up", "far_up", "dua_down", "far_down"]:
    zoningmods[col] = np.nan

zoningmods.loc[zoningmods.zoningmodcat.str.contains("gg"), "gg_id"] = "GG"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("exd"), "exd_id"] = "exd"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("tra1"), "tra_id"] = "TRA1"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("tra2"), "tra_id"] = "TRA2"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("tra3"), "tra_id"] = "TRA3"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("hra"), "hra_id"] = "HRA"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("ppa"), "ppa_id"] = "PPA"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("ugb"), "ugb_id"] = "UGB"
zoningmods.loc[zoningmods.zoningmodcat.str.contains("uninc_ugb"), "ugb_id"] = "Uninc_UGB"


Add Zoning Step 1: Add unincorporated UGB zoning (doesn't apply to PPAs)

In [5]:
# if within UGB but in unincorporated area, some upzoning is allowed for historic expansion
# Non-Residential building on parcel
zoningmods.loc[(zoningmods.ugb_id == 'Uninc_UGB') & (zoningmods.exd_id != 'exd'), 'dua_up'] = 2.75
zoningmods.loc[(zoningmods.ugb_id == 'Uninc_UGB') & (zoningmods.exd_id != 'exd'), 'add_bldg'] = 'HS'
# Residential building on parcel
zoningmods.loc[(zoningmods.ugb_id == 'Uninc_UGB') & (zoningmods.exd_id == 'exd'), 'dua_up'] = 1.5
zoningmods.loc[(zoningmods.ugb_id == 'Uninc_UGB') & (zoningmods.exd_id == 'exd'), 'add_bldg'] = 'HS'

Step 3: Add residential and non-residential growth geography upzoning, which can override unincorporated UGB zoning

In [6]:
# update "dua_up", "far_up", and "add_bldg" with PBA50+ residential and non-residential zoning

# GG + TRA1 + HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA1') & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 200
zoningmods.loc[zm, 'far_up'] = 10
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + TRA1 + HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA1') & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 50
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + TRA1 + Non-HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA1') & (zoningmods.hra_id != 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 150
zoningmods.loc[zm, 'far_up'] = 10
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + TRA1 + Non=HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA1') & (zoningmods.hra_id != 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 50
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + TRA2 + HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA2') & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 100
zoningmods.loc[zm, 'far_up'] = 5
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + TRA2 + HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA2') & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 50
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + TRA2 + Non-HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA2') & (zoningmods.hra_id != 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 75
zoningmods.loc[zm, 'far_up'] = 5
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + TRA2 + Non-HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA2') & (zoningmods.hra_id != 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + TRA3 + HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA3') & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 50
zoningmods.loc[zm, 'far_up'] = 2
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + TRA3 + HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA3') & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 50
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + TRA3 + Non-HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA3') & (zoningmods.hra_id != 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'far_up'] = 2
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + TRA3 + Non-HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id == 'TRA3') & (zoningmods.hra_id != 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + Non-TRA + HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id.isnull()) & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + Non-TRA + HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id.isnull()) & (zoningmods.hra_id == 'HRA') & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

# GG + Non-TRA + Non-HRA + Non-Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id.isnull()) & (zoningmods.hra_id.isnull()) & (zoningmods.exd_id != 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'add_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# GG + Non-TRA + Non-HRA + Residential
zm = (zoningmods.gg_id == 'GG') & (zoningmods.tra_id.isnull()) & (zoningmods.hra_id.isnull()) & (zoningmods.exd_id == 'exd')
zoningmods.loc[zm, 'dua_up'] = 35
zoningmods.loc[zm, 'add_bldg'] = 'HM, MR'

Step 3: Add PPA zoning, which can override unincorporated UGB zoning and growth geography upzoning

In [7]:
# if within PPA, add industrial zoning and "drop_bldg" for all other building types
zoningmods.loc[zoningmods.ppa_id == 'PPA', 'far_up'] = 2
zoningmods.loc[zoningmods.ppa_id == 'PPA', 'add_bldg'] = 'IW'
zoningmods.loc[zoningmods.ppa_id == 'PPA', 'drop_bldg'] = 'HT, HM, OF, HO, SC, RS, RB, MR, MT, ME'
# also set "dua_up" and "add_bldg" to nan
zoningmods.loc[zoningmods.ppa_id == 'PPA', 'dua_up'] = np.nan
zoningmods.loc[zoningmods.ppa_id == 'PPA', 'add_bldg'] = np.nan


Step 4: Add UGB zoning, which can override unincorporated UGB, growth geography, and PPA upzoning

In [8]:
# if outside of UGB "far_down" and "dua_down" are 0
zoningmods.loc[zoningmods.ugb_id.isnull(), 'dua_down'] = 0
zoningmods.loc[zoningmods.ugb_id.isnull(), 'far_down'] = 0
# and also set other "dua_up", "far_up", and "add_bldg" to nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'far_up'] = np.nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'dua_up'] = np.nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'add_bldg'] = np.nan
zoningmods.loc[zoningmods.ugb_id.isnull(), 'drop_bldg'] = np.nan


In [9]:
# export
zoningmods.to_csv("M:/urban_modeling/baus/BAUS Inputs/plan_strategies/zoning_mods_PBA50Plus_DBP.csv", index=False)